In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import requests
import os
from datetime import datetime

In [2]:

pd.set_option('display.max_columns', 500)

def call_endpoint_with_players_and_teams(url, max_level=3):
    """
    Fetches data from the PrizePicks API and includes player names and their team names.
    
    Parameters:
        url (str): The API endpoint to call.
        max_level (int): The level of JSON normalization to apply.
    
    Returns:
        pd.DataFrame: A DataFrame containing the main data with player names and team names.
    """
    # Fetch the API response
    resp = requests.get(url).json()
    
    # Normalize the main data
    data = pd.json_normalize(resp['data'], max_level=max_level)
    
    # Normalize the included data
    included = pd.json_normalize(resp['included'], max_level=max_level)
    
    # Filter for player data in the included section
    players = included[included['type'] == 'new_player'][['id', 'attributes.name', 'attributes.team', 'attributes.team_name']].rename(
        columns={
            'id': 'player_id', 
            'attributes.name': 'player_name', 
            'attributes.team': 'team_abbreviation',
            'attributes.team_name': 'team_name'
        })
    
    # Merge the player names and team names into the main data
    data = pd.merge(data, players, how='left', left_on=['relationships.new_player.data.id'], right_on=['player_id'])
    
    return data

# API URL for NBA projections
url = 'https://partner-api.prizepicks.com/projections?league_id=7&per_page=1000'

# Call the function and fetch data with player names and team names
df = call_endpoint_with_players_and_teams(url)

# Display the first few rows
df.head(2)


,type,id,attributes.adjusted_odds,attributes.board_time,attributes.custom_image,attributes.description,attributes.discount_name,attributes.discount_percentage,attributes.end_time,attributes.flash_sale_line_score,attributes.game_id,attributes.hr_20,attributes.in_game,attributes.is_live,attributes.is_promo,attributes.line_score,attributes.odds_type,attributes.projection_type,attributes.rank,attributes.refundable,attributes.start_time,attributes.stat_display_name,attributes.stat_type,attributes.status,attributes.today,attributes.trending_count,attributes.tv_channel,attributes.updated_at,relationships.duration.data.type,relationships.duration.data.id,relationships.game.data.type,relationships.game.data.id,relationships.league.data.type,relationships.league.data.id,relationships.new_player.data.type,relationships.new_player.data.id,relationships.projection_type.data.type,relationships.projection_type.data.id,relationships.score.data,relationships.stat_average.data.type,relationships.stat_average.data.id,relationships.stat_type.data.type,relationships.stat_type.data.id,player_id,player_name,team_abbreviation,team_name
0,projection,3725880,None,2025-01-23T13:00:00-05:00,None,BOS,flash_sale,26.0,2025-01-23T22:10:00-05:00,16.5,NBA_game_3K3AEUfkqJWLCcSB5ysjTuYg,True,False,False,True,22.5,standard,Single Stat,0,True,2025-01-23T22:10:00-05:00,Points,Points,pre_game,True,28738.0,None,2025-01-23T13:00:21-05:00,duration,11,game,42688,league,7,new_player,188012,projection_type,2,None,stat_average,80154,stat_type,19,188012,LeBron James,LAL,Lakers
1,projection,3719646,None,2025-01-01T10:27:00-05:00,None,SAS/IND,NaN,NaN,None,NaN,NBA_game_uwDUOX7iDY3m1yVe7iG1T5CT,True,False,False,False,49.5,standard,Single Stat,1,True,2025-01-23T14:10:00-05:00,Points (Combo),Points (Combo),pre_game,True,NaN,None,2025-01-23T10:57:37-05:00,duration,11,game,42490,league,7,new_player,175890,projection_type,2,None,NaN,NaN,stat_type,425,175890,Pascal Siakam + Victor Wembanyama,IND/SAS,None


In [3]:
# Select the desired columns from the DataFrame
o_df = df[['player_name', 'team_abbreviation', 'attributes.stat_type', 'attributes.line_score']].copy()
o_df = o_df[~o_df['attributes.stat_type'].str.contains(r'\(Combo\)', na=False)]
o_df = o_df[~o_df['attributes.stat_type'].str.contains('Fantasy Score', na=False)]


In [4]:
o_df 

,player_name,team_abbreviation,attributes.stat_type,attributes.line_score
0,LeBron James,LAL,Points,22.5
28,Andrew Nembhard,IND,Assists,4.5
29,Andrew Nembhard,IND,Pts+Rebs+Asts,19.5
30,Andrew Nembhard,IND,Rebounds,3.5
31,Andrew Nembhard,IND,Pts+Rebs,14.5
...,...,...,...,...
2507,Kyle Kuzma,WAS,Pts+Rebs+Asts,19.5
2508,Carlton Carrington,WAS,3-PT Made,0.5
2509,Kawhi Leonard,LAC,Points,14.5
2510,Kevin Porter Jr.,LAC,Pts+Rebs,19.5


In [5]:
o_df = o_df.rename(columns={'player_name': 'Player', 'team_abbreviation': 'Team', 'attributes.stat_type': 'Stat', 'attributes.line_score': 'Line'})
o_df
current_date = datetime.now().strftime('%m-%d-%Y')
o_df.to_csv(f'prizepicks_nba_{current_date}.csv', index=False)